In [ ]:
%matplotlib ipympl

# Standard library

# Third-party.
import matplotlib.pyplot as plt
import numpy as np

# Local library
from microhhpy.spatial import Domain, plot_domains

TF = np.float32

## Domain setup without spatial projection.

In [ ]:
d0 = Domain(
    xsize=3200,
    ysize=3200,
    itot=32,
    jtot=32,
    n_ghost=3,
    n_sponge=4,
    )

d1 = Domain(
    xsize=1600,
    ysize=1600,
    itot=32,
    jtot=32,
    n_ghost=3,
    n_sponge=4,
    parent=d0,
    center_in_parent=True,
    )

d2 = Domain(
    xsize=800,
    ysize=800,
    itot=32,
    jtot=32,
    n_ghost=3,
    n_sponge=0,
    parent=d1,
    xstart_in_parent=200,
    ystart_in_parent=200,
    )

# Domain start/end should be an integer multiple of the parent dx and dy.
# This won't work:
#d2 = Domain(
#    xsize=800,
#    ysize=800,
#    itot=32,
#    jtot=32,
#    n_ghost=3,
#    n_sponge=0,
#    parent=d1,
#    xstart_in_parent=np.pi**np.e,
#    ystart_in_parent=np.pi**np.e,
#    )

d0.child = d1
d1.child = d2

plot_domains([d0, d1, d2])

## Domain setup with spatial projection.

In [ ]:
lat = 51.97
lon = 4.92

# Define projection lat/lon <-> meters.
#proj_str = '+proj=utm +zone=31 +ellps=intl +towgs84=-87,-98,-121,0,0,0,0 +units=m +no_defs +type=crs'
proj_str = f'+proj=lcc +lat_1={lat-1} +lat_2={lat+1} +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

d0 = Domain(
    xsize=512_000,
    ysize=512_000,
    itot=64,
    jtot=64,
    n_ghost=3,
    n_sponge=5,
    lon=lon,
    lat=lat,
    anchor='center',
    proj_str=proj_str
    )

d1 = Domain(
    xsize=256_000,
    ysize=256_000,
    itot=64,
    jtot=64,
    n_ghost=3,
    n_sponge=3,
    parent=d0,
    center_in_parent=True
    )

d0.child = d1

plot_domains([d0, d1], use_projection=True, scatter_lonlat=False)

## Spatial transforms

Each domain has `.proj` instance that contains the lat/lon coordinates (see docstring of `microhhpy.spatial.Projection`), and can do x/y <-> lon/lat transforms.



In [ ]:
d0 = Domain(
    xsize=32_000,
    ysize=32_000,
    itot=4,
    jtot=4,
    n_ghost=3,
    n_sponge=5,
    lon=lon,
    lat=lat,
    anchor='center',
    proj_str=proj_str
    )

plt.figure()
plt.plot(d0.proj.lon_u, d0.proj.lat_u, 'r+')
plt.plot(d0.proj.lon_v, d0.proj.lat_v, 'b+')
plt.plot(d0.proj.lon, d0.proj.lat, 'gx')
plt.plot(d0.proj.bbox_lon, d0.proj.bbox_lat, 'k--')

x,y = d0.proj.to_xy(4.92, 51.97)
lon,lat = d0.proj.to_lonlat(x, y)

print(f'Coordinates ({lon:.2f}, {lat:.2f}) degrees = ({x:.2f}, {y:.2f}) m in LES.')